In [1]:
from functions.load_training_data import training_samples
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import torch
import time

# Load training samples

In [2]:
training_images, training_labels = training_samples(num_samples=1000, colors=[1,1,1,1])

# Configure GPU for training

In [17]:
device=torch.device("cuda:4")

In [18]:
device.index

4

In [20]:
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=float)
y = torch.randn(N, D_out, device=device, dtype=float)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=float)
w2 = torch.randn(H, D_out, device=device, dtype=float)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 338.16037028284325
199 1.1945154497838004
299 0.009261975034062198
399 9.622170803078146e-05
499 1.1015726401816325e-06


# Create Model

In [48]:
from torch import nn
model = torch.nn.Sequential(nn.Conv2d(in_channels=64, out_channels=32, kernel_size=(1,1),
                                     stride=(2,2)),
                            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=(1,1),
                                     stride=(2,2)),
                            nn.Conv2d(in_channels=16, out_channels=3, kernel_size=(1,1),
                                      stride=(2,2)),
                            nn.Flatten(),
                            nn.Linear(256, 128),
                            nn.Linear(24, 4)
                           )

In [27]:
img_tensor = torch.Tensor(list(training_images))
label_tensor = torch.Tensor(list(training_labels))

In [49]:
predictions = model(img_tensor).numpy()

RuntimeError: size mismatch, m1: [4000 x 24], m2: [256 x 128] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:136